**Homework 5**

Take the abstracts of the first 20 papers in the given URL, translate them into Chinese, and write them into the file named 'my abstract.txt'.

URL = https://openaccess.thecvf.com/ICCV2021?day=2021-10-12

In [11]:
import requests
from bs4 import BeautifulSoup
import re
import random
from hashlib import md5

def getUrl(url):
    response = requests.get(url)
    pdfUrl = []
    count = 0
    bs = BeautifulSoup(response.text,"html.parser")
    pdfs = bs.find_all(href=re.compile('/content/.*html'))
    names = []
    for tag in pdfs:
        if count < 20: 
            pdfUrl.append("https://openaccess.thecvf.com"+tag.get("href"))
            count = count+1
            names.append(re.split('[./]', tag.get("href"))[-2][:-16].replace("_"," "))
        else: break
    return names,pdfUrl


def getAbstract(paperUrls):
    abstract = []
    for url in paperUrls:
        response = requests.get(url)
        bs = BeautifulSoup(response.text,"html.parser")
        abstract.append(bs.find(id="abstract").text.strip())
    return abstract

def translate_api(input_text):
    # Set your own appid/appkey.
    appid = '20231227001923466'
    appkey = 'h_QCzh_iFjBIJtd5hiQQ'
    def make_md5(s, encoding='utf-8'):
        return md5(s.encode(encoding)).hexdigest()
    # For list of language codes, please refer to `https://api.fanyi.baidu.com/doc/21`
    from_lang = 'en'
    to_lang = 'zh'
    endpoint = 'http://api.fanyi.baidu.com'
    path = '/api/trans/vip/translate'
    url = endpoint + path
    
    query = input_text
    # Generate salt and sign
    salt = random.randint(32768, 65536)
    sign = make_md5(appid + query + str(salt) + appkey)
    # Build request
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    payload = {'appid': appid, 'q': query, 'from': from_lang, 'to': to_lang, 'salt': salt, 'sign': sign}
    # Send request
    r = requests.post(url, params=payload, headers=headers)
    result = r.json()
    return result['trans_result'][0]['dst']

def write2txt(path,title,contents):
    with open(path,"w") as f:
        for ti,con in zip(title,contents):
            f.write(ti+"\n")
            f.write(con+"\n")
            f.write("\n")

In [12]:
web = "https://openaccess.thecvf.com/ICCV2021?day=2021-10-12"
name,pdfurls = getUrl(web)
abstracts = getAbstract(pdfurls)
ch_abstracts = []

In [13]:
for ab in abstracts:
    ch_abstracts.append(translate_api(ab))
write2txt("./myabstract.txt",name,ch_abstracts)

In [6]:
len(abstracts)

20

In [8]:
abstracts[1]

'Continually learning in the real world must overcome many challenges, among which noisy labels are a common and inevitable issue. In this work, we present a replay-based continual learning framework that simultaneously addresses both catastrophic forgetting and noisy labels for the first time. Our solution is based on two observations; (i) forgetting can be mitigated even with noisy labels via self-supervised learning, and (ii) the purity of the replay buffer is crucial. Building on this regard, we propose two key components of our method: (i) a self-supervised replay technique named Self-Replay, which can circumvent erroneous training signals arising from noisy labeled data, and (ii) the Self-Centered filter that maintains a purified replay buffer via centrality-based stochastic graph ensembles. The empirical results on MNIST, CIFAR-10, CIFAR-100, and WebVision with real-world noise demonstrate that our framework can maintain a highly pure replay buffer amidst noisy streamed data whi